In [1]:
import time, logging
from pathlib import Path
from flofish.experiment import Experiment
from flofish.image import Image

2025-02-26 11:50:41,334	[INFO]     omnipose/gpu.py               <module>....()	 line   9	On ARM, OMP_NUM_THREADS set to 1


omnipose installed


### Download test data

In [2]:
from napari_flofish import get_smRNAfish_Ecoli_rpoD_rnlAB_hipBA
get_smRNAfish_Ecoli_rpoD_rnlAB_hipBA(Path("tests/data/exp16/expected"), cleanup=False)

### Read in microscope images

In [ ]:
exp = Experiment.from_cfg_file("tests/data/exp16/config.json")
exp.inputdir = "tests/data/exp16/input"
exp.outputdir = "tests/data/exp16/output-notebook"
exp.create_image_list()

2025-02-26 11:50:52,356	[INFO]     flofish/experiment.py         from..._file()	 line  30	found cfg file tests/data/exp16/config.json
2025-02-26 11:50:52,358	[INFO]                                   crea..._list()	 line  43	getting .vsi files from tests/data/exp16/input
2025-02-26 11:50:52,359	[INFO]                                               	 line  46	examining image tests/data/exp16/input/MG1655_GLU_OD_0.3_left_CY5, CY3.5 NAR, CY3, DAPI_02.vsi
2025-02-26 11:50:52,360	[INFO]                                               	 line  52	..found VSI directory _MG1655_GLU_OD_0.3_left_CY5, CY3.5 NAR, CY3, DAPI_02_
2025-02-26 11:50:52,361	[INFO]                                               	 line  67	..found DIC file MG1655_GLU_OD_0.3_left_DIC_02.tif
2025-02-26 11:50:52,362	[INFO]                                               	 line  73	..all good, adding image to image list


In [4]:
for params in exp.images.values():
    logging.info(params)
    my_image = Image.from_dict(params, exp)

    tic = time.time()
    my_image.read_image()
    my_image.read_cells()
    my_image.align()
    my_image.create_grgb()

    # save image (write to dir)
    my_image.save_input_layers()
    my_image.time['01-configure'] = time.time() - tic
    my_image.save_metadata("configure")

2025-02-26 11:50:56,191	[INFO]     ipykernel_46355/441914148.py  <module>....()	 line   2	{'vsi_file': 'MG1655_GLU_OD_0.3_left_CY5, CY3.5 NAR, CY3, DAPI_02.vsi', 'cell_file': 'MG1655_GLU_OD_0.3_left_DIC_02.tif', 'valid': True}
2025-02-26 11:50:56,193	[INFO]     flofish/image.py              __init__....()	 line  77	created output dir tests/data/exp16/output/MG1655_GLU_OD_0.3_left_02
2025-02-26 11:50:56,194	[INFO]                                   read_image..()	 line 102	reading fluorescence image: MG1655_GLU_OD_0.3_left_CY5, CY3.5 NAR, CY3, DAPI_02.vsi


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


2025-02-26 11:50:57,825	[INFO]                                               	 line 104	..found scenes: ('CY5, CY3.5 NAR, CY3, DAPI', 'macro image')
2025-02-26 11:50:58,612	[INFO]                                               	 line 110	....reading scene 'CY5, CY3.5 NAR, CY3, DAPI' (shape (1, 4, 31, 2304, 2304))
2025-02-26 11:50:58,613	[INFO]                                               	 line 111	....found channels ['CY5', 'CY3.5 NAR', 'CY3', 'DAPI']
2025-02-26 11:50:58,613	[INFO]                                               	 line 118	......reading in channel: C=0 filter=CY5 mrna=hipBA
2025-02-26 11:51:24,624	[INFO]                                               	 line 118	......reading in channel: C=1 filter=CY3.5 NAR mrna=rnlAB
2025-02-26 11:51:24,625	[INFO]                                               	 line 118	......reading in channel: C=2 filter=CY3 mrna=rpoD
2025-02-26 11:51:24,625	[INFO]                                               	 line 118	......reading in channel: C=3 

### Segment images

In [5]:
exp.read_image_list_from_jsons()
exp.init_omnipose()

for f in exp.json_files:
    logging.info(f'image: {f}')
    my_image = Image.from_json(f, exp)

    # segment image (~ 02-segment)
    tic = time.time()
    my_image.segment_cells()
    my_image.time['02-segment-cells'] = time.time() - tic

    tic = time.time()
    my_image.segment_dapi()
    my_image.time['02-segment-dapi'] = time.time() - tic

    # postprocess masks
    tic = time.time()
    my_image.postprocess_masks()
    my_image.time['02-segment-pp'] = time.time() - tic

    # indicate in metadata which stage was run last
    my_image.save_metadata("segment")

2025-02-26 11:59:53,584	[INFO]     flofish/experiment.py         read...jsons()	 line  83	reading json file tests/data/exp16/output/MG1655_GLU_OD_0.3_left_02/img.json
2025-02-26 11:59:53,594	[INFO]                                               	 line  83	reading json file tests/data/exp16/output/MG1655_GLU_OD_0.3_left_02/img.json
2025-02-26 11:59:53,595	[INFO]                                   init...ipose()	 line 105	reading json file tests/data/exp16/output/MG1655_GLU_OD_0.3_left_02/img.json
2025-02-26 11:59:53,596	[INFO]     flofish/image.py              __init__....()	 line  83	found output dir tests/data/exp16/output/MG1655_GLU_OD_0.3_left_02
pretrained_model False cyto2_omni
2025-02-26 11:59:53,596	[INFO]     cellpose_omni/models.py                   	 line 435	>>cyto2_omni<< model set to be used
2025-02-26 11:59:53,598	[INFO]     cellpose_omni/core.py         assi...evice()	 line  67	Using GPU.
2025-02-26 11:59:53,690	[INFO]                                   __init__....()	 line

/Users/adele/Projects/omnipose/cellpose_omni/transforms.py:587: UserWarning: chan to seg' has value range of ZERO
  warnings.warn("chan to seg' has value range of ZERO")


2025-02-26 12:00:00,577	[INFO]                                               	 line 279	Found 2345 cells
2025-02-26 12:00:00,600	[INFO]                                               	 line 289	writing cell mask to tests/data/exp16/output/MG1655_GLU_OD_0.3_left_02/DIC_masks.model=cyto2_omni_chan=[1,2]_diameter=0_minsize=200_mask=0_flow=0.tif
2025-02-26 12:00:00,605	[INFO]                                   segment_dapi()	 line 294	segmenting DAPI image
pretrained_model False nuclei
2025-02-26 12:00:00,631	[INFO]     cellpose_omni/models.py       __init__....()	 line 435	>>nuclei<< model set to be used
2025-02-26 12:00:00,632	[INFO]     cellpose_omni/core.py         assi...evice()	 line  67	Using GPU.
2025-02-26 12:00:00,716	[INFO]                                   __init__....()	 line 163	u-net config: ([2, 32, 64, 128, 256], 3, 2)


/Users/adele/PycharmProjects/flofish/flofish/image.py:285: UserWarning: /Users/adele/PycharmProjects/flofish/flofish/tests/data/exp16/output/MG1655_GLU_OD_0.3_left_02/DIC_masks.model=cyto2_omni_chan=[1,2]_diameter=0_minsize=200_mask=0_flow=0.tif is a low contrast image
  io.imsave(maskfile_latest, mask)


2025-02-26 12:00:05,052	[INFO]     flofish/image.py              segment_dapi()	 line 330	Found 2972 cells
2025-02-26 12:00:05,072	[INFO]                                               	 line 342	writing DAPI mask to tests/data/exp16/output/MG1655_GLU_OD_0.3_left_02/DAPI_masks_model=nuclei_chan=[0,0]_diameter=0.0_minsize=10_mask=0.0_flow=0.0.tif
2025-02-26 12:00:05,077	[INFO]                                   post...masks()	 line 346	postprocessing masks
2025-02-26 12:00:05,077	[INFO]                                               	 line 348	..removing cell masks with no DAPI


/Users/adele/PycharmProjects/flofish/flofish/image.py:338: UserWarning: /Users/adele/PycharmProjects/flofish/flofish/tests/data/exp16/output/MG1655_GLU_OD_0.3_left_02/DAPI_masks_model=nuclei_chan=[0,0]_diameter=0.0_minsize=10_mask=0.0_flow=0.0.tif is a low contrast image
  io.imsave(maskfile_latest, mask)


removing 11 cells with no dapi signal
2025-02-26 12:00:31,432	[INFO]                                               	 line 351	..discarding masks of size < 200
2025-02-26 12:00:31,789	[INFO]                                               	 line 355	..discarding large, round masks (min size 1000, min excentricity 0.8
2025-02-26 12:00:31,954	[INFO]                                               	 line 362	..computing expanded masks (by 3)


/Users/adele/PycharmProjects/flofish/flofish/image.py:360: UserWarning: /Users/adele/PycharmProjects/flofish/flofish/tests/data/exp16/output/MG1655_GLU_OD_0.3_left_02/DIC_masks_pp.tif is a low contrast image
  io.imsave(cellmaskfile, self.cell_masks_by_area)


2025-02-26 12:00:32,196	[INFO]                                   save...adata()	 line 201	saving metadata to file: tests/data/exp16/output/MG1655_GLU_OD_0.3_left_02/img.json


/Users/adele/PycharmProjects/flofish/flofish/image.py:366: UserWarning: /Users/adele/PycharmProjects/flofish/flofish/tests/data/exp16/output/MG1655_GLU_OD_0.3_left_02/DIC_masks_pp_expanded.tif is a low contrast image
  io.imsave(cellmaskfile, self.cell_masks_expanded)


### Detect spots